# Análisis de Datos Financieros con SQL

**Introducción**

En este proyecto realizaremos un análisis exploratorio y financiero sobre los datos transaccionales de cuentas bancarias extraídos desde archivos bancarios consolidados. Este análisis tiene como propósito identificar patrones, detectar transacciones inusuales, categorizar operaciones, y obtener indicadores clave sobre los movimientos financieros.

Usaremos SQL ejecutado en Python (Google Colab) para procesar esta base de datos, la cual fue convertida a un formato SQLite o similar, y que originalmente proviene de un archivo Excel

**Introduction**

In this project, we will conduct an exploratory and financial analysis of bank account transaction data extracted from consolidated bank statements.
The main purpose of this analysis is to:

Identify transaction patterns, Detect unusual or suspicious activities,
Categorize transactions, Generate key financial indicators based on bank operations.

We will use SQL queries executed within Python (Google Colab) to process this dataset, which was originally provided in Excel format:

# Descripción del Análisis

El análisis estará enfocado en responder preguntas clave sobre:

1. Volumen y frecuencia de transacciones.

2. Identificación de cuentas con mayor actividad.

3. Clasificación de transacciones por tipo de operación (retiros, depósitos).

4. Detección de posibles operaciones sospechosas (altos montos, transacciones repetidas).

5. Análisis de saldo y su evolución en el tiempo.

**Variables Principales:**

Account No. → Número de cuenta involucrada.

Date → Fecha de la transacción.

Transaction Details → Detalle o narración de la transacción.

Cheque No. → Número de cheque (si aplica).

Value Date → Fecha efectiva de la transacción.

Withdrawal Amount → Monto retirado.

Deposit Amount → Monto depositado.

Balance Amount → Saldo disponible en la cuenta.

# Analysis Description

This analysis will focus on answering key questions regarding:

1. Transaction volume and frequency.

2. Identification of the most active accounts.

3. Classification of transactions by type (withdrawals or deposits).

4. Detection of potentially suspicious transactions (high amounts, repetitive transfers).

5. Evolution of account balances over time.

**Main Variables:**

Account No. → Bank account number involved in the transaction.

Date → Date of the transaction.

Transaction Details → Transaction description or narration from the bank statement.

Cheque No. → Cheque number (if applicable).

Value Date → Date when the transaction was completed.

Withdrawal Amount → Amount withdrawn from the account.

Deposit Amount → Amount deposited into the account.

Balance Amount → Current balance after the transaction.

**Preguntas del Análisis.**

¿Cuántas transacciones totales hay en la base de datos?

¿Cuál es la suma total de depósitos y retiros en la base de datos?

¿Cuáles son las 5 cuentas con mayor cantidad de transacciones registradas?

¿Cuáles son las 5 transacciones con mayores montos de depósito?

¿Cuáles son las 5 transacciones con mayores montos de retiro?

¿Qué cuentas tienen saldo promedio más alto? (Top 5 por saldo promedio).

¿Cuántas transacciones involucran transferencias internas (filtrar por 'INTERNAL FUND TRANSFER' en 'Transaction Details')?

¿Qué días tienen mayor volumen de transacciones? (Contar por fecha).

¿Hay transacciones duplicadas? (Mismo monto, misma fecha, misma cuenta y similar narración).

¿Cuántas transacciones podrían considerarse sospechosas por tener montos superiores a un umbral definido? (Ejemplo: > 500,000).

----

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import sqlite3

# Ruta del archivo Excel
file_path = '/content/drive/MyDrive/Databases/Financial and Bank/Bank Transaction Data/bank.xlsx'

# Leer Excel (asegúrate de que la hoja correcta sea la primera o especifica el nombre con sheet_name='Sheet1')
df = pd.read_excel(file_path)

# Crear conexión a base de datos SQLite (archivo temporal en Colab)
conn = sqlite3.connect('/content/bank_transactions.db')

# Cargar DataFrame en SQLite (la tabla se llamará 'bank')
df.to_sql('bank', conn, if_exists='replace', index=False)

print("Database loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Database loaded successfully!


# ¿Cuántas transacciones totales hay en la base de datos?

In [2]:
query = "SELECT COUNT(*) AS Total_Transactions FROM bank"
result = pd.read_sql_query(query, conn)
print(result)

   Total_Transactions
0              116201


El dataset contiene un total de 116,201 transacciones bancarias registradas

# ¿Cuál es la suma total de depósitos y retiros en la base de datos?

In [6]:
query = """
SELECT
    SUM([WITHDRAWAL AMT]) AS Total_Withdrawals,
    SUM([DEPOSIT AMT]) AS Total_Deposits
FROM bank
"""
result = pd.read_sql_query(query, conn)
print(result)

   Total_Withdrawals  Total_Deposits
0       2.403916e+11    2.384902e+11


El total de retiros registrados en las transacciones es aproximadamente 240.39 mil millones.

El total de depósitos es aproximadamente 238.49 mil millones.

La diferencia entre retiros y depósitos es leve, pero los retiros superan a los depósitos en este conjunto de datos.

Esto podría indicar:

Altos volúmenes de operaciones de salida.

Necesidad de revisar la naturaleza de las transacciones (operaciones normales, préstamos, pagos, etc.).

# ¿Cuáles son las 5 cuentas con mayor cantidad de transacciones registradas?

In [7]:
query = """
SELECT [Account No], COUNT(*) AS Transaction_Count
FROM bank
GROUP BY [Account No]
ORDER BY Transaction_Count DESC
LIMIT 5
"""
result = pd.read_sql_query(query, conn)
print(result)


      Account No  Transaction_Count
0       1196428'              48779
1  409000362497'              29840
2  409000438620'              13454
3       1196711'              10536
4  409000493210'               6014


La cuenta 1196428 es, por mucho, la que más transacciones tiene en la base de datos (casi 49,000 transacciones), lo cual sugiere que podría tratarse de:

Una cuenta operativa principal.

Una cuenta que agrupa o canaliza múltiples operaciones.

Las siguientes cuentas también muestran altos volúmenes, aunque con diferencias marcadas respecto a la primera.

Este tipo de cuentas deben analizarse con más detalle para detectar:

Si concentran movimientos repetitivos.

Si están asociadas a patrones de transferencias internas o externos.

También puede ser útil analizar el saldo promedio o los montos promedio de estas cuentas más adelante.

# ¿Cuáles son las 5 transacciones con mayores montos de depósito?

In [8]:
query = """
SELECT [Account No], [DATE], [TRANSACTION DETAILS], [DEPOSIT AMT]
FROM bank
ORDER BY [DEPOSIT AMT] DESC
LIMIT 5
"""
result = pd.read_sql_query(query, conn)
print(result)

      Account No                 DATE               TRANSACTION DETAILS  \
0  409000438620'  2016-02-25 00:00:00         GOLD LABEL INVESTMENTS (P   
1       1196711'  2017-04-28 00:00:00         909000039501 Draw Down Cr   
2       1196711'  2017-05-29 00:00:00         Loan Payment Reversal For   
3       1196711'  2018-10-31 00:00:00  TRF FROM  Indiaforensic SERVICES   
4       1196428'  2016-10-13 00:00:00         NEFT/CITIN16697927307/PAY   

    DEPOSIT AMT  
0  5.448000e+08  
1  5.000000e+08  
2  4.482072e+08  
3  3.540000e+08  
4  2.119594e+08  


Estas transacciones representan altísimos montos de depósito, todas superiores a 200 millones.

Las transacciones involucran:

Inversiones (ej.: GOLD LABEL INVESTMENTS).

Préstamos / Draw Down y Reversiones de Pago de Préstamos.

Transferencias internas relevantes como la de Indiaforensic SERVICES.

Algunas cuentas aparecen más de una vez, lo que sugiere que manejan operaciones de gran escala o financieras.

Estas transacciones podrían merecer análisis adicionales, ya que pueden:

Ser parte de operaciones totalmente legítimas (préstamos, inversiones).

Representar movimientos que requieren monitoreo por posibles actividades de blanqueo de capitales.

# ¿Cuáles son las 5 transacciones con mayores montos de retiro?


In [9]:
query = """
SELECT [Account No], [DATE], [TRANSACTION DETAILS], [WITHDRAWAL AMT]
FROM bank
ORDER BY [WITHDRAWAL AMT] DESC
LIMIT 5
"""
result = pd.read_sql_query(query, conn)
print(result)

      Account No                 DATE        TRANSACTION DETAILS  \
0       1196711'  2018-06-26 00:00:00  909000055389 - Payoff Sou   
1       1196711'  2017-05-26 00:00:00  Loan Recovery For90900003   
2  409000438620'  2016-03-08 00:00:00       YOURSELF FOR M Joshi   
3  409000425051'  2018-10-31 00:00:00         TRF TO  Myur Joshi   
4       1196711'  2015-06-20 00:00:00  TAKEOVR FUNDS OF OD FACIL   

   WITHDRAWAL AMT  
0    4.594475e+08  
1    4.482072e+08  
2    4.000000e+08  
3    3.540000e+08  
4    2.671403e+08  


Estas transacciones muestran los retiros más grandes registrados en el dataset, con montos superiores a 267 millones.

Las descripciones reflejan transacciones de:

Pago de préstamos o “Payoff”.

Recuperación de préstamos.

Transferencias hacia personas específicas (ej.: "Myur Joshi" o "M Joshi").

Movimientos de fondos relacionados con “Overdraft Facility” (OD Facility).

La cuenta 1196711' aparece en varias de estas transacciones de alto valor, lo que confirma que es una cuenta de alta actividad financiera y gran magnitud.

Estas transacciones podrían estar relacionadas con:

Liquidaciones de deudas.

Reestructuraciones financieras.

# ¿Qué cuentas tienen saldo promedio más alto?

In [10]:
query = """
SELECT [Account No], AVG([BALANCE AMT]) AS Average_Balance
FROM bank
GROUP BY [Account No]
ORDER BY Average_Balance DESC
LIMIT 5
"""
result = pd.read_sql_query(query, conn)
print(result)

      Account No  Average_Balance
0  409000611074'     1.478073e+06
1  409000493201'     9.981640e+05
2  409000425051'    -4.703390e+06
3  409000405747'    -4.766824e+08
4  409000438620'    -5.294276e+08


Solo 2 cuentas presentan saldos promedio positivos:

409000611074' con un saldo promedio de ~1.47 millones.

409000493201' con casi 1 millón.

Las otras 3 cuentas tienen saldo promedio negativo, incluyendo:

La cuenta 409000438620' con un saldo promedio negativo muy elevado (-529 millones).

Esto puede indicar cuentas sobregiradas o relacionadas con productos financieros como créditos o líneas de financiamiento con saldos negativos.

Esta información es clave en un análisis financiero:

Puedes identificar las cuentas que están en riesgo o con mayor apalancamiento financiero.

También es útil para monitorear cuentas que podrían ser inusuales por sus saldos negativos recurrentes.

# ¿Cuántas transacciones involucran transferencias internas?


In [11]:
query = """
SELECT COUNT(*) AS Internal_Transfers
FROM bank
WHERE [TRANSACTION DETAILS] LIKE '%INTERNAL FUND TRANSFER%'
"""
result = pd.read_sql_query(query, conn)
print(result)

   Internal_Transfers
0                3271


Se identificaron 3,271 transacciones relacionadas con transferencias internas según el campo TRANSACTION DETAILS.

Esto representa un volumen importante de movimientos dentro de la organización o entre cuentas propias del banco.

Estas transferencias internas pueden incluir:

Movimientos entre cuentas del mismo cliente.

Liquidaciones de productos financieros internos.

Ajustes contables o de saldos entre cuentas.

Este dato es muy relevante en análisis de:

Auditoría interna.

Detección de fraude o blanqueo de capitales (especialmente si los montos son altos y repetitivos).

Identificación de operaciones "mirror" o de cancelación.

# ¿Qué días tienen mayor volumen de transacciones?

In [12]:
query = """
SELECT [DATE], COUNT(*) AS Transaction_Count
FROM bank
GROUP BY [DATE]
ORDER BY Transaction_Count DESC
LIMIT 5
"""
result = pd.read_sql_query(query, conn)
print(result)

                  DATE  Transaction_Count
0  2017-07-27 00:00:00                567
1  2018-08-13 00:00:00                463
2  2017-11-08 00:00:00                402
3  2017-10-07 00:00:00                382
4  2018-07-10 00:00:00                374


El día con mayor volumen de transacciones fue el 27 de julio de 2017 con 567 operaciones.

Otros días con alta actividad corresponden a los años 2017 y 2018.

Estos picos pueden deberse a:

Cierres contables o de ciclo financiero.

Pagos masivos (como nóminas, dividendos o liquidaciones).

Movimientos inusuales que podrían requerir revisión.

Este tipo de análisis es muy útil para identificar:

Fechas clave de actividad operativa.

Comportamientos cíclicos o recurrentes.

# ¿Hay transacciones duplicadas?


In [13]:
query = """
SELECT [Account No], [DATE], [TRANSACTION DETAILS], [WITHDRAWAL AMT], [DEPOSIT AMT], COUNT(*) AS Occurrences
FROM bank
GROUP BY [Account No], [DATE], [TRANSACTION DETAILS], [WITHDRAWAL AMT], [DEPOSIT AMT]
HAVING Occurrences > 1
ORDER BY Occurrences DESC
"""
result = pd.read_sql_query(query, conn)
print(result)

         Account No                 DATE              TRANSACTION DETAILS  \
0          1196428'  2018-01-23 00:00:00       FDRL/NATIONAL ELECTRONIC F   
1          1196428'  2018-05-10 00:00:00       FDRL/NATIONAL ELECTRONIC F   
2          1196428'  2018-05-10 00:00:00       FDRL/NATIONAL ELECTRONIC F   
3     409000362497'  2015-11-16 00:00:00  TRF FROM Indiaforensic SERVICES   
4     409000362497'  2016-01-11 00:00:00  TRF FROM Indiaforensic SERVICES   
...             ...                  ...                              ...   
5940  409000611074'  2019-01-03 00:00:00       FDRL/INTERNAL FUND TRANSFE   
5941  409000611074'  2019-01-09 00:00:00       FDRL/INTERNAL FUND TRANSFE   
5942  409000611074'  2019-01-11 00:00:00       FDRL/INTERNAL FUND TRANSFE   
5943  409000611074'  2019-01-14 00:00:00       FDRL/INTERNAL FUND TRANSFE   
5944  409000611074'  2019-01-19 00:00:00       FDRL/INTERNAL FUND TRANSFE   

      WITHDRAWAL AMT  DEPOSIT AMT  Occurrences  
0             1250.0      

Se detectaron transacciones duplicadas:

El reporte muestra un total de 5,945 combinaciones duplicadas (mismo número de cuenta, fecha, narración, monto de retiro y depósito).

Algunas transacciones se repiten decenas de veces:

Por ejemplo, una transacción con retiro de 1,250.0 aparece 60 veces en la cuenta 1196428'.

Otras como transferencias internas y depósitos recurrentes también aparecen duplicadas.

Las transacciones duplicadas incluyen:

Pagos electrónicos nacionales (FDRL/NATIONAL ELECTRONIC F).

Transferencias internas frecuentes.

Transferencias repetidas desde "Indiaforensic SERVICES".

Estas duplicaciones pueden deberse a:

Pagos masivos o automatizados que se registran por separado en la base.

Errores de consolidación de datos o carga repetida.

Transacciones fragmentadas que tienen la misma narración y montos.

# ¿Cuántas transacciones podrían considerarse sospechosas por tener montos superiores a un umbral definido?

In [14]:
threshold = 500000  # Umbral definido

query = f"""
SELECT COUNT(*) AS Suspicious_Transactions
FROM bank
WHERE [WITHDRAWAL AMT] > {threshold} OR [DEPOSIT AMT] > {threshold}
"""
result = pd.read_sql_query(query, conn)
print(result)

   Suspicious_Transactions
0                    49193


Hay un total de 49,193 transacciones que superan el umbral definido de 500,000 (ya sea en retiros o depósitos).

Esto representa una proporción significativa del total de transacciones (116,201 registros en total), aproximadamente un 42% del dataset.

Estas transacciones son candidatas para:

Análisis detallado por parte de equipos de auditoría o compliance.

Revisión como parte de programas de prevención de lavado de dinero o fraude financiero.

Este tipo de consultas es crucial para construir reglas automáticas de monitoreo en sistemas bancarios.

# Conclusiones Finales del Análisis Financiero de Transacciones Bancarias

1. Alto volumen de transacciones

  El análisis reveló un total de 116,201 transacciones registradas, lo cual confirma que estamos trabajando con un dataset bancario robusto, adecuado para análisis de patrones financieros, fraude y comportamiento transaccional.

2. Movimiento financiero significativo
  Durante el análisis, detectamos montos totales muy elevados:

  Retiros totales: 240.39 mil millones.

  Depósitos totales: 238.49 mil millones.

  Aunque ambos valores son similares, los retiros superan ligeramente a los depósitos, lo que puede indicar un mayor flujo de salida en las cuentas analizadas.

3. Concentración de transacciones en pocas cuentas

  Las transacciones están fuertemente concentradas en algunas cuentas:

  La cuenta con mayor actividad registró casi 49,000 transacciones.

  Este hallazgo sugiere que ciertas cuentas son operativas o de alto tráfico, lo cual es común en cuentas empresariales o nodos financieros clave.

4. Transacciones de montos elevados recurrentes

  Se identificaron depósitos y retiros individuales por cientos de millones, vinculados a:

  Préstamos, liquidaciones o inversiones.

  Transferencias internas de grandes montos.
  Estas operaciones podrían ser legítimas pero deben ser monitoreadas debido al tamaño y repetición de los montos.

5. Saldos negativos elevados en algunas cuentas

  Varias cuentas registraron saldos promedio negativos muy altos, incluso de cientos de millones en negativo, lo cual sugiere:

  Cuentas sobregiradas.

  Operaciones apalancadas o deudoras.
  Esto puede ser un foco de riesgo financiero y debe analizarse más a fondo.

6. Alta cantidad de transferencias internas

  Se detectaron 3,271 transacciones relacionadas con transferencias internas.
  Este tipo de operaciones es normal en instituciones financieras, pero también puede ocultar prácticas de compensación de fondos o flujos circulares.

7. Fechas con picos inusuales de actividad

  Algunos días específicos registraron picos de transacciones, como el 27 de julio de 2017 con 567 transacciones.
  Estas fechas clave deben ser revisadas, ya que pueden relacionarse con:

  Eventos financieros específicos.

  Pagos masivos.

  Potenciales anomalías o incidentes.

8. Presencia de transacciones duplicadas

  Se encontraron 5,945 transacciones duplicadas basadas en combinación de cuenta, fecha, monto y descripción.
  Esto puede deberse a:

  Pagos automáticos repetidos.

  Errores de carga o consolidación.

  Actividades sospechosas que requieren validación adicional.

9. Alto número de transacciones potencialmente sospechosas

  Se identificaron 49,193 transacciones con montos superiores a un umbral de 500,000, lo cual representa aproximadamente el 42% del dataset.
  Estas transacciones deben ser monitoreadas bajo políticas de prevención de:

  Lavado de activos.

  Financiamiento ilícito.

  Fraudes financieros.